In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
%matplotlib inline  

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
mypath = '/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data/unsupervised'
df = pd.read_csv(mypath + '/clustering_data.csv')
comps = pd.read_csv(mypath + '/competitors.csv')

## Truncate data to those that have funding

In [8]:
#get rid of companies that don't have fundings
path = '/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data/csv_export'
orgs = pd.read_csv(path + '/organizations.csv')

/Users/Lucy/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
orgs.loc[0]

company_name                                                   Soko Glam
primary_role                                                     company
domain                                                      sokoglam.com
homepage_url                                     http://www.sokoglam.com
country_code                                                         USA
state_code                                                            NY
region                                                     New York City
city                                                            New York
zipcode                                                              NaN
address                                                              NaN
status                                                         operating
short_description      An eCommerce beauty shop that thoughtfully cur...
category_list                                beauty|cosmetics|e-commerce
category_group_list                 commerce and sh

In [12]:
new_df = pd.merge(df,orgs[['uuid','funding_rounds','funding_total_usd','status']],left_on = 'company_uuid',right_on = 'uuid')
trunc_df = new_df[new_df.funding_rounds > 1].copy()
trunc_df.to_csv('/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data/unsupervised/trunc_clustering.csv')

In [7]:
trunc_df.description = trunc_df.description.astype(str)

## Training happens in mercer, codes are in Codes/Mercer/baseline_tfidf_cosine.py

## get scores

In [10]:
scores = pd.read_csv(mypath+'/tfidf_cosine.csv', index_col = 0)

In [11]:
scores.shape

(771, 26648)

In [82]:
scores.head()

,0,1,2,3,4,5,6,7,8,9,...,26638,26639,26640,26641,26642,26643,26644,26645,26646,26647
0,1.000000,0.047483,0.028005,0.00000,0.099057,0.012608,0.045231,0.026240,0.029096,0.016908,...,0.000000,0.000000,0.175115,0.062487,0.028624,0.043187,0.049553,0.073281,0.098872,0.084553
1,0.047483,1.000000,0.000000,0.00000,0.000000,0.099297,0.051765,0.000000,0.016598,0.163185,...,0.018278,0.000000,0.000000,0.000000,0.039795,0.000000,0.100059,0.000000,0.000000,0.027681
2,0.028005,0.000000,1.000000,0.00000,0.014761,0.014027,0.000000,0.027749,0.008982,0.036216,...,0.033041,0.000000,0.080125,0.000000,0.000000,0.027288,0.014297,0.000000,0.014626,0.034625
3,0.000000,0.000000,0.000000,1.00000,0.035690,0.020541,0.000000,0.177743,0.133287,0.000000,...,0.000000,0.150964,0.066061,0.000000,0.000000,0.000000,0.000000,0.147680,0.000000,0.000000
4,0.099057,0.000000,0.014761,0.03569,1.000000,0.010790,0.000000,0.081143,0.020435,0.000000,...,0.000000,0.000000,0.000000,0.016937,0.034041,0.000000,0.010724,0.020971,0.000000,0.062986


In [55]:
#trunc_df = trunc_df.reset_index()
trunc_df = trunc_df.drop(trunc_df.columns[[0]],1)

### airbnb

In [57]:
trunc_df[trunc_df.company_name == 'Airbnb']

,company_uuid,company_name,company_category_list,category_group_list,short_description,description,founded_on,uuid,funding_rounds,funding_total_usd,status
94,bcb617c3-9e43-d5b0-1d14-82b795f2642f,Airbnb,Hospitality|Marketplace|Sharing Economy|Travel...,commerce and shopping|travel and tourism,Airbnb is an online community marketplace for ...,Airbnb is a community marketplace for people t...,2008-08-01,bcb617c3-9e43-d5b0-1d14-82b795f2642f,10,3.950282e+09,operating


In [58]:
airbnb = scores.loc[94]
airbnb_10 = airbnb.argsort()[::-1][:11]

In [59]:
list(airbnb_10)

[94, 3811, 18692, 22436, 23470, 9864, 1962, 23216, 11638, 16077, 14183]

In [61]:
trunc_df.loc[list(airbnb_10)]

,company_uuid,company_name,company_category_list,category_group_list,short_description,description,founded_on,uuid,funding_rounds,funding_total_usd,status
94,bcb617c3-9e43-d5b0-1d14-82b795f2642f,Airbnb,Hospitality|Marketplace|Sharing Economy|Travel...,commerce and shopping|travel and tourism,Airbnb is an online community marketplace for ...,Airbnb is a community marketplace for people t...,2008-08-01,bcb617c3-9e43-d5b0-1d14-82b795f2642f,10,3.950282e+09,operating
3811,6a2859a6-e096-926a-ee81-871ccbd16da8,Slack,Collaboration|Enterprise Software|Messaging|Re...,information technology|internet services|messa...,Slack is a team communication application prov...,Slack is a team communication application prov...,2009-01-01,6a2859a6-e096-926a-ee81-871ccbd16da8,9,5.399500e+08,operating
18692,69ccf811-5714-d23c-054c-96648ff9d800,Duda,Developer Tools|Internet|Mobile|Web Design,design|internet services|mobile|software,Duda is an online DIY website development plat...,Duda is an online DIY website development plat...,2009-04-23,69ccf811-5714-d23c-054c-96648ff9d800,3,1.860000e+07,operating
22436,59800a12-290f-76e9-153b-5315b9c71aed,Qik,Messaging|Mobile|Video,information technology|internet services|media...,Qik is an online messenger that allows its use...,Qik is an online messenger that allows its use...,2006-06-01,59800a12-290f-76e9-153b-5315b9c71aed,5,1.660000e+07,closed
23470,5b22b8e6-6227-f1dc-09e1-451bb5032fbc,CitizenNet,Advertising|Social Media,advertising|internet services|media and entert...,CitizenNet is a Facebook preferred marketing p...,CitizenNet is a [Facebook](https://www.crunchb...,2008-01-01,5b22b8e6-6227-f1dc-09e1-451bb5032fbc,3,7.999960e+05,operating
9864,3524c45c-ff03-f813-36e9-284d2d42c55a,IgnitionOne,Advertising|Digital Media|Marketing Automation,advertising|media and entertainment|sales and ...,IgnitionOne is a digital marketing company tha...,IgnitionOne is a digital marketing company tha...,2004-01-01,3524c45c-ff03-f813-36e9-284d2d42c55a,4,6.821321e+07,operating
1962,53f1e559-f724-92b1-a8a8-15cf257d6462,Quikly,E-Commerce|Retail|Social Media,commerce and shopping|internet services|media ...,"Quikly, a marketing platform, helps retailers ...",Quikly is a promotional marketing platform des...,2012-01-01,53f1e559-f724-92b1-a8a8-15cf257d6462,6,5.613396e+06,operating
23216,6fb702b5-f41c-9d7b-21be-44a995dc6f6e,Heroku,Enterprise Software|Venture Capital|Web Develo...,financial services|lending and investments|sof...,Heroku is a multi-language cloud application p...,Heroku is a multi-language cloud application p...,2007-07-01,6fb702b5-f41c-9d7b-21be-44a995dc6f6e,3,1.302000e+07,acquired
11638,01d43739-3964-5b75-6a74-73374a3ef5d8,SNAP Interactive,Apps|Mobile|Social Network|Software,apps|internet services|mobile|software,SNAP Interactive develops online dating and so...,SNAP Interactive develops online dating and so...,2005-07-19,01d43739-3964-5b75-6a74-73374a3ef5d8,3,1.225000e+07,ipo
16077,a302ceb6-0f9a-6d7f-1e36-e35145e9b4b9,Kanvas Labs,Lifestyle|Mobile|Photography|Photo Sharing|Soc...,community and lifestyle|content and publishing...,Kanvas Labs is building a mobile platform that...,Kanvas Labs is building a mobile platform that...,2010-01-01,a302ceb6-0f9a-6d7f-1e36-e35145e9b4b9,5,3.436500e+06,acquired


### Civatech Oncology

In [85]:
top_10 = scores.loc[168].argsort()[::-1][:11]
trunc_df.loc[list(top_10)]

,company_uuid,company_name,company_category_list,category_group_list,short_description,description,founded_on,uuid,funding_rounds,funding_total_usd,status
168,682e8b5a-e26e-87ed-e894-c9d928f0dc34,Civatech Oncology,Biotechnology,biotechnology|science and engineering,"Civatech Oncology, a medical device company, d...",Civatech Oncology is an emerging medical devic...,2006-01-01,682e8b5a-e26e-87ed-e894-c9d928f0dc34,9,7066814.0,operating
20475,b81f02a0-9168-b300-1bc5-f034034cbac8,ImmuVen,Biotechnology|Life Science|Pharmaceutical,biotechnology|health care|science and engineering,ImmuVen develops drugs to treat infectious dis...,ImmuVenis is a biotechnology company with nove...,2007-01-01,b81f02a0-9168-b300-1bc5-f034034cbac8,3,1930000.0,closed
6563,bbf7caed-cb22-fbb7-8c6a-049e54ac3d41,Gynesonics,Health Care|Hospital|Medical,health care,Gynesonics develops an ablation device for tre...,"Gynesonics Inc., developer of an ablation devi...",2005-01-01,bbf7caed-cb22-fbb7-8c6a-049e54ac3d41,10,117296441.0,operating
12696,7dbaf0da-8539-809c-bd0d-5646dee4a08a,lensgen,Health Care,health care,Lensgen creates adaptive intraocular lenses fo...,Creating a new generation adaptive introcular ...,2011-08-01,7dbaf0da-8539-809c-bd0d-5646dee4a08a,3,2749995.0,operating
6115,0733e3d4-48e3-4f12-6dfb-da537ac98928,MicroTransponder,Health Care|Medical|Medical Device,health care,"MicroTransponder, a medical device company, de...",MicroTransponder's lead researcher discusses t...,2007-01-01,0733e3d4-48e3-4f12-6dfb-da537ac98928,4,13579068.0,operating
21132,0fe5e7d3-98d0-a322-3286-23500fa52590,Acceleron Pharma,Biotechnology|Health Care|Medical,biotechnology|health care|science and engineering,Acceleron is a biopharmaceutical company engag...,"Acceleron Pharma, Inc. operates as a biopharma...",2004-01-01,0fe5e7d3-98d0-a322-3286-23500fa52590,6,113701127.0,ipo
24028,1fd33849-b28b-cbae-89b3-888abe08c97b,Interlace Medical,Health Care|Medical|Medical Device,health care,Interlace Medical develops devices for interve...,"Interlace Medical, Inc. is focused on designin...",2006-01-01,1fd33849-b28b-cbae-89b3-888abe08c97b,2,23000000.0,acquired
23028,37d84b73-1aad-ad54-1914-f2abfce459e2,Xoft,Advanced Materials|Electronics|Medical Device,consumer electronics|hardware|health care|manu...,Xoft develops electronic brachytherapy (eBx) s...,"Xoft, Inc. develops electronic brachytherapy (...",1998-01-01,37d84b73-1aad-ad54-1914-f2abfce459e2,6,100496138.0,acquired
13980,b168da22-a3c4-16f1-8547-8dd4b0f2d4cd,ALDEA Pharmaceuticals,Biotechnology|Medical Device|Pharmaceutical,biotechnology|health care|science and engineering,ALDEA Pharmaceuticals develops small-molecule ...,ALDEA Pharmaceuticals is developing small-mole...,2011-01-01,b168da22-a3c4-16f1-8547-8dd4b0f2d4cd,4,41997183.0,operating
13346,26345765-af1f-e385-54be-fef125cc4d15,Sanuthera,Health Care|Medical|Medical Device,health care,"Sanuthera, Inc. designs and manufactures medic...","Sanuthera, Inc. designs and manufactures medic...",2011-01-01,26345765-af1f-e385-54be-fef125cc4d15,2,440697.0,operating


### AppNexus

In [94]:
top_10_3 = scores.loc[7].argsort()[::-1][:11]
trunc_df.loc[list(top_10_3)]

,company_uuid,company_name,company_category_list,category_group_list,short_description,description,founded_on,uuid,funding_rounds,funding_total_usd,status
7,86b95953-4e19-100b-236f-15bd88f3eb6f,AppNexus,Advertising|Internet|Service Industry,advertising|internet services|sales and marketing,AppNexus is a technology company whose cloud-b...,AppNexus is a technology company whose cloud-b...,2007-09-01,86b95953-4e19-100b-236f-15bd88f3eb6f,9,3.191719e+08,operating
2660,667b7125-60e1-6cd7-f944-719ccbfe3a2d,Kiosked,Advertising|Advertising Platforms|Digital Mark...,advertising|internet services|sales and marketing,Kiosked is the world’s leading Advertising Aut...,"Kiosked is the leading, independent Advertisin...",2010-01-01,667b7125-60e1-6cd7-f944-719ccbfe3a2d,6,7.689245e+07,operating
342,ba3ae1b8-6ed6-f121-6666-40b7bf52a54c,Beeswax,Advertising,advertising|sales and marketing,Beeswax is an advertising technology platform ...,Beeswax is an advertising technology platform ...,2014-09-01,ba3ae1b8-6ed6-f121-6666-40b7bf52a54c,2,1.100000e+07,operating
24004,a77b73cb-b580-2b01-417a-3d331a5fb775,Pudding Media,Advertising|Advertising Platforms|Marketing|Te...,advertising|hardware|sales and marketing,Pudding Media is a mobile advertising company ...,Pudding Media provides turnkey mobile advertis...,2006-01-01,a77b73cb-b580-2b01-417a-3d331a5fb775,3,1.450000e+07,operating
15577,26d9607a-4159-3a84-74c8-711f7724922f,Genesis Media,Advertising|Predictive Analytics|Publishing|SE...,advertising|content and publishing|data and an...,Genesis Media is the only platform making adve...,Genesis Media’s real-time analytics platform i...,2011-01-01,26d9607a-4159-3a84-74c8-711f7724922f,3,1.700000e+07,operating
20226,9fd34d1d-dddb-4a79-3a90-1791144e5f54,Todacell,Ad Network|Advertising,advertising|sales and marketing,Todacell provides proprietary ad flow manageme...,Todacell is the an off-portal mobile display a...,2007-01-01,9fd34d1d-dddb-4a79-3a90-1791144e5f54,4,4.350000e+06,closed
24832,5b626988-b378-2803-5571-fbf222bab82b,Adjug,Ad Network|Advertising|Internet|Publishing,advertising|content and publishing|internet se...,Adjug is an online banner exchange marketplace...,AdJug is an online advertising marketplace. It...,2007-01-01,5b626988-b378-2803-5571-fbf222bab82b,2,8.500000e+06,acquired
4629,10e37a49-4f4d-9c66-94a7-dd0bcd85fab4,ResponsiveAds,Ad Network|Advertising,advertising|sales and marketing,ResponsiveAds is the responsive design company...,"ResponsiveAds, Inc. is the publishing and adve...",2012-01-01,10e37a49-4f4d-9c66-94a7-dd0bcd85fab4,3,3.900000e+05,operating
24586,1eabbf59-fb5e-2d86-1dce-656219230cfb,BiggerBoat,Ad Network|Advertising,advertising|sales and marketing,BiggerBoat is an online advertising network fo...,BiggerBoat is an online advertising network th...,2003-01-01,1eabbf59-fb5e-2d86-1dce-656219230cfb,2,7.000000e+06,closed
7584,6b8046fa-6cd6-7c50-6609-cecda723cded,PLYmedia,Ad Network|Advertising|Advertising Platforms|I...,advertising|commerce and shopping|information ...,PLYmedia is a next-generation ad-tech company ...,PLYmedia is a next-generation ad-tech company ...,2006-01-01,6b8046fa-6cd6-7c50-6609-cecda723cded,3,5.700000e+06,operating


### Skymind

In [95]:
top_10_4 = scores.loc[20].argsort()[::-1][:11]
trunc_df.loc[list(top_10_4)]

,company_uuid,company_name,company_category_list,category_group_list,short_description,description,founded_on,uuid,funding_rounds,funding_total_usd,status
20,10297137-abdd-3e65-6031-f89499a94a30,Skymind,Analytics|Artificial Intelligence|Big Data|Bus...,data and analytics|information technology|scie...,"Skymind makes tools for prediction, data analy...",Skymind is a business intelligence and enterpr...,2014-01-01,10297137-abdd-3e65-6031-f89499a94a30,3,3320000.0,operating
14007,564818f6-0120-f53a-29b2-3141507cbb5e,Alfresco,Document Management|Enterprise|Enterprise Soft...,information technology|software,Alfresco is an open source model providing ent...,Alfresco is the leading open source alternativ...,2005-01-01,564818f6-0120-f53a-29b2-3141507cbb5e,4,68500000.0,operating
19669,e0e183dc-b8d2-986c-1f0e-6bd66ab0e368,Zenoss,Enterprise Software|Information Technology|IT ...,information technology|software,Zenoss is a leading provider of unified monito...,Zenoss is a commercial open source software co...,2005-01-01,e0e183dc-b8d2-986c-1f0e-6bd66ab0e368,4,45632042.0,operating
3136,9baeba98-b4e8-d19e-811a-5718f329ce57,Assembla,Collaboration|Software|Ticketing,events|media and entertainment|software,Assembla provides online tools for distributed...,Assembla provides an integrated set of online ...,2005-01-01,9baeba98-b4e8-d19e-811a-5718f329ce57,2,375000.0,operating
25529,d02d521e-56d5-eec3-51bb-38f2590f55ef,SourceLabs,Linux|Open Source|Software|Web Hosting,internet services|platforms|software,SourceLabs is a provider of a set of tools and...,SourceLabs is a provider of a set of tools and...,2004-01-01,d02d521e-56d5-eec3-51bb-38f2590f55ef,2,10700000.0,closed
21107,2c0ec385-3e11-533e-7b09-205400f51791,VNT Software Ltd,Enterprise Software,software,VNT Software Ltd is a provider of automatic bu...,VNT Software Ltd is a provider of automatic bu...,2010-05-01,2c0ec385-3e11-533e-7b09-205400f51791,2,3100000.0,operating
26607,e8aa2634-9c75-f946-4e8c-3264902c0a90,Lutris Technologies,Open Source,software,"Lutris Technologies, an Open Source enterprise...","Lutris Technologies, an Open Source enterprise...",1995-01-01,e8aa2634-9c75-f946-4e8c-3264902c0a90,2,10000000.0,closed
9983,303e64c0-a645-0234-c57d-00e9cd76efa3,Black Duck Software,Cyber Security|Open Source|Software,information technology|privacy and security|so...,Black Duck secures & manages open source softw...,Organizations worldwide use Black Duck product...,2002-01-01,303e64c0-a645-0234-c57d-00e9cd76efa3,8,75500000.0,operating
5400,c80a36a4-7299-08d3-8666-1e3adc720ed3,Luxe RV,Leisure|Tourism|Travel,community and lifestyle|travel and tourism,Luxe RV is an on demand luxury RV rental service.,Luxury RV Rentals in Los Angeles - San Francis...,2015-02-20,c80a36a4-7299-08d3-8666-1e3adc720ed3,2,2250000.0,operating
18455,2c860e3b-7b71-ecda-83a6-6b4c41cbbd12,BonitaSoft,Business Development|Open Source|Software,software,BonitaSoft offers an open-source BPM and workf...,BonitaSoft is a software company created by th...,2009-01-01,2c860e3b-7b71-ecda-83a6-6b4c41cbbd12,3,27000000.0,operating


In [101]:
sorted(scores.loc[168],reverse = True)

[1.0,
 0.41514259392800001,
 0.412315959984,
 0.41002320886400001,
 0.40080637300300004,
 0.40024011652900005,
 0.395875565569,
 0.38921847423699996,
 0.38516612203299999,
 0.38412517692699999,
 0.37192204556100006,
 0.370526904983,
 0.36621012698399996,
 0.365237922662,
 0.36520582264200002,
 0.36446770439900006,
 0.363124628758,
 0.359839783468,
 0.35921078434100001,
 0.35789185606000001,
 0.355664265162,
 0.35437487100000004,
 0.35393788577600005,
 0.35139826125899998,
 0.34444162698500003,
 0.34440327168200002,
 0.34248362809499999,
 0.34238377764099998,
 0.34166004714599996,
 0.33792214618100003,
 0.33341603035900003,
 0.33210077478,
 0.33086605186000001,
 0.33078813328099999,
 0.330214274044,
 0.329960007,
 0.32966438131600001,
 0.32786128867800002,
 0.32698871620800002,
 0.32605822580499999,
 0.32601649206,
 0.32476564375,
 0.32457045969300002,
 0.324402453062,
 0.32181131229900001,
 0.321753662444,
 0.32053489491600001,
 0.320428388294,
 0.32037682410399998,
 0.3199272768140000

In [105]:
trunc_df.loc[list(top_10)].to_csv('Civatech_Oncology.csv')

In [106]:
trunc_df.loc[168].description

"Civatech Oncology is an emerging medical device company focused on the development of novel polymer based devices that treat cancer using localized, low-dose-rate radiation.\r\n\r\nBecause polymers are easy to manufacture in different shapes and formats, CivaTech's patented and patent-pending technology enables devices that can be designed and fabricated for specific tumor types. Our first product, the CivaString, has been granted FDA 510k clearance to treat any solid tumor.\r\n\r\nWe are developing other products that are specifically designed to treat lung and breast cancer."